In [1]:
!pip install wandb -qqq
!pip install pytorch-lightning

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 143 kB 42.1 MB/s 
     |████████████████████████████████| 97 kB 5.9 MB/s 
     |████████████████████████████████| 180 kB 46.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 527 kB 5.3 MB/s 
     |████████████████████████████████| 952 kB 39.4 MB/s 
     |████████████████████████████████| 133 kB 60.5 MB/s 
     |████████████████████████████████| 596 kB 54.2 MB/s 
     |████████████████████████████████| 396 kB 48.7 MB/s 
     |████████████████████████████████| 829 kB 41.8 MB/s 
     |████████████████████████████████| 1.1 MB 30.6 MB/s 
     |████████████████████████████████| 271 kB 41.4 MB/s 
     |████████████████████████████████| 144 kB 43.9 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=9b7a2ea77fca35cb7b283c4395bf017a5f8acb7e961a3eab49b

In [2]:
import numpy as np
import seaborn as sn
import random
import math
from torch import autograd,Tensor
import matplotlib.pyplot as plt
from torch.types import Device
from tqdm import tqdm
import argparse
import copy
import sklearn
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, RandomSampler, SequentialSampler, random_split, DataLoader, IterableDataset, ConcatDataset
import torch.utils
import torch.utils.checkpoint

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MAP_SIZE = 15
MODEL_SAVE_PATH = "/content/drive/MyDrive/DPST-MLRC/Planner/dspt"
CHECKPOINT_FOLDER = "/content/drive/MyDrive/DPST-MLRC/Planner/checkpoints"
filename = str(MAP_SIZE) + '_baseline_testing.npz'

In [5]:
if MAP_SIZE == 15 :
  config = {
      "RANDOM_SEED": 42,

      "train_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/trainy15.npz',
      "train_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/trainx15.npz',

      "test_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/testy15.npz',
      "test_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/testx15.npz',

      "val_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/valy15.npz',
      "val_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/valx15.npz',

      "batch_size": 20,
      "epochs": 40,
      "learning_rate": 1,

      "train_dataset_size": 100000,
      "test_dataset_size": 5000,
      "val_dataset_size": 5000,
  }
elif MAP_SIZE == 30 :
  config = {
      "RANDOM_SEED": 42,

      "train_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/trainy30.npz',
      "train_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/trainx30.npz',

      "test_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/testy30.npz',
      "test_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/testx30.npz',

      "val_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/valy30.npz',
      "val_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/valx30.npz',

      "batch_size": 20,
      "epochs": 40,
      "learning_rate": 1,

      "train_dataset_size": 100000,
      "test_dataset_size": 5000,
      "val_dataset_size": 5000,
  }
elif MAP_SIZE == 50 :
  config = {
      "RANDOM_SEED": 42,

      "train_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/trainy50.npz',
      "train_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/trainx50.npz',

      "test_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/testy50.npz',
      "test_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/testx50.npz',

      "val_output_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/valy50.npz',
      "val_input_dest_path": '/content/drive/MyDrive/DPST-MLRC/synthetic_navigation_datasets/valx50.npz',

      "batch_size": 2, #we'll use gradient accumulation for the 50 X 50 maps
      "epochs": 40,
      "learning_rate": 1,

      "train_dataset_size": 100000,
      "test_dataset_size": 5000,
      "val_dataset_size": 5000,
  }



In [7]:
x_file=config['train_input_dest_path']
y_file=config['train_output_dest_path']

In [8]:
# np_x = np.load(x_file)
# np_y = np.load(y_file)

# x_list = np.array([np_x[i] for i in np_x.files])
# y_list = np.array([np_y[i] for i in np_y.files])

In [9]:
# mazes = np.concatenate((x_list, x_list, x_list), 0)

In [ ]:
train_x_file=config['train_input_dest_path']
train_y_file=config['train_output_dest_path']

val_x_file=config['val_input_dest_path']
val_y_file=config['val_output_dest_path']

test_x_file=config['test_input_dest_path']
test_y_file=config['test_output_dest_path']

In [ ]:
np_x = np.load(train_x_file)
np_y = np.load(train_y_file)

train_mazes = np.array([np_x[i][0] for i in np_x.files])
train_goal_maps = np.array([np_x[i][1] for i in np_x.files])
train_opt_policies = np.array([np_y[i] for i in np_y.files])

np_x = np.load(val_x_file)
np_y = np.load(val_y_file)

valid_mazes = np.array([np_x[i][0] for i in np_x.files])
valid_goal_maps = np.array([np_x[i][1] for i in np_x.files])
valid_opt_policies = np.array([np_y[i] for i in np_y.files])

np_x = np.load(test_x_file)
np_y = np.load(test_y_file)

test_mazes = np.array([np_x[i][0] for i in np_x.files])
test_goal_maps = np.array([np_x[i][1] for i in np_x.files])
test_opt_policies = np.array([np_y[i] for i in np_y.files])

In [12]:
# for i in np_x.files:
#   print(np_x[i][1])
#   break


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
# Re-shuffle
mazes = np.concatenate((train_mazes, valid_mazes, test_mazes), 0)
goal_maps = np.concatenate(
    (train_goal_maps, valid_goal_maps, test_goal_maps), 0)
opt_policies = np.concatenate(
    (train_opt_policies, valid_opt_policies, test_opt_policies), 0)

shuffidx = np.random.permutation(mazes.shape[0])
mazes = mazes[shuffidx]
goal_maps = goal_maps[shuffidx]
opt_policies = opt_policies[shuffidx]

train_mazes = mazes[:config['train_dataset_size']]
train_goal_maps = goal_maps[:config['train_dataset_size']]
train_opt_policies = opt_policies[:config['train_dataset_size']]

valid_mazes = mazes[config['train_dataset_size']:config['train_dataset_size'] + config['val_dataset_size']]
valid_goal_maps = goal_maps[config['train_dataset_size']:config['train_dataset_size'] + config['val_dataset_size']]
valid_opt_policies = opt_policies[config['train_dataset_size']:config['train_dataset_size'] + config['val_dataset_size']]

test_mazes = mazes[config['train_dataset_size'] + config['val_dataset_size']:]
test_goal_maps = goal_maps[config['train_dataset_size'] + config['val_dataset_size']:]
test_opt_policies = opt_policies[config['train_dataset_size'] + config['val_dataset_size']:]

# Save to numpy
np.savez_compressed(filename, train_mazes, train_goal_maps,
                    train_opt_policies, valid_mazes, valid_goal_maps,
                    valid_opt_policies, test_mazes, test_goal_maps,
                    test_opt_policies)